In [1]:
import sys
sys.path.append('../sgmm')
sys.path.append('../metrics')
sys.path.append('../loaders')
sys.path.append('../Misc')
sys.path.append('../visual')
sys.path.append('../otherModels')
sys.path.append('../LogOdds')

In [2]:
import numpy as np
import pandas as pd
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from supervisedBmm import SupervisedBMM
from metricsFunctions import calc_metrics, CalculateSoftLogReg, optimalTau,metrics_cluster,sgmmResults
# from loaders2 import loader
# from mlModels import logisticRegressionCv2, neural_nets, randomforests,\
# kmeansLogRegr, xboost, gradboost,kmeansBNB
# from sklearn.naive_bayes import BernoulliNB
# from utility import entropy,asymm_entropy
# from ftest_logodds import ftest_uncorr
# from ftest_logodds import restest
# #from clustmap import plotclustmap
# from clustmap_newborn_kmeans_noopt import plotclustmap
# from sklearn.linear_model import LogisticRegression

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sparcs = pd.read_csv("~/data/CDPHP/xiao/SPARCS_Subsets/Obsolete/sparcs25%Circ_DeHos_Outflow_Region.csv") 

#sparcs = sparcs.sample(frac=0.02, random_state = 1512)

d_circ_tr, d_circ_te = train_test_split(sparcs, test_size=0.2, random_state = 1512)

print(d_circ_tr.shape[0], d_circ_tr.shape[1])

39888 301


In [5]:
# metric matrix columns
columns = ['cluster', 'size', 'high_cost%','low_cost%', 
                       'TP', 'TN', 'FP', 'FN', 
                       'FPR', 'specificity', 'sensitivity', 'precision',
                       'accuracy', 'balanced accuracy', 'f1', 'auc']

In [6]:
# feature list
features = list(sparcs.iloc[:,:-1])

In [7]:
#Preparing the data
Xtrain, Xtest = d_circ_tr.iloc[:,0:-1].values, d_circ_te.iloc[:,0:-1].values
ytrain, ytest = d_circ_tr.iloc[:,-1].values.astype(int), d_circ_te.iloc[:,-1].values.astype(int)

In [9]:
import time
import random 

train, val = train_test_split(d_circ_tr, test_size = 0.2, random_state = 1512)
y_train, y_val = train.iloc[:,-1], val.iloc[:,-1]

print( train.shape, val.shape )

#SET SBMM PARAMETERS

max_iter = 30
max_iter2 = 30

train_np = train.iloc[:,0:-1].values
val_np = val.iloc[:,0:-1].values

y_train_np = y_train.values
y_val_np = y_val.values

test_re_B = []
train_re_B = []
seed_list_B = []

start = time.time()

for j in range(0,10): 

    seed_num = random.randint(0,100000)
    seed_list_B.append(seed_num)
    print("\n TRAINING WITH RANDOM SEED {}". format(seed_num))

    #test clusters 2-10
    for i in range(3,10):
        print("\n\n\nTRAINING WITH {} CLUSTERS".format(i))
        np.random.seed( seed = seed_num)
        modelB = SupervisedBMM(max_iter = max_iter, max_iter2 = max_iter2, n_clusters = i)
        modelB = modelB.fitB(Xtrain = train_np, Xtest=val_np, ytrain = y_train_np)

        logisRegre = modelB.LogRegr
        fitP = modelB.fitParams
        labTrainB  = fitP['labTrain']
        mTest = modelB.predict_BMMS(val_np)
        labTestB = np.argmax( mTest, axis = 1 )

        probTest, probTrain = modelB.predict_prob_int_B( Xtest = val_np, Xtrain = train_np )
        tau = optimalTau(probTrain, y_train_np)
        metTest,_ = calc_metrics(custom_prob = probTest.copy(), tau = tau, y = y_val_np)
        metTrain ,_= calc_metrics(custom_prob = probTrain.copy(), tau = tau, y = y_train_np)
        metTestSBMM = pd.DataFrame( [metTest], columns = columns)
        metTrainSBMM = pd.DataFrame( [metTrain], columns = columns)

        test_re_B.append( metTestSBMM )
        train_re_B.append( metTrainSBMM )

        end = time.time() - start
print("time elapsed: {}".format( end ))

(31910, 301) (7978, 301)

 TRAINING WITH RANDOM SEED 362



TRAINING WITH 3 CLUSTERS
BMM iteration: 0, error: 0.14323903363085008
BMM iteration: 1, error: 0.04130213968131621
BMM iteration: 2, error: 0.09207892837675052
BMM iteration: 3, error: 0.14396884517021302
BMM iteration: 4, error: 0.12377423676415245
BMM iteration: 5, error: 0.10017913834159713
BMM iteration: 6, error: 0.06886264115430338
BMM iteration: 7, error: 0.0360901315133833
BMM iteration: 8, error: 0.01528024664422644
BMM iteration: 9, error: 0.00572538678228251
BMM iteration: 10, error: 0.002205952153550269
BMM iteration: 11, error: 0.0012238938013671749
BMM iteration: 12, error: 0.0010924981849238118
BMM iteration: 13, error: 0.0010525333819395408
BMM iteration: 14, error: 0.0009045529664612165



TRAINING WITH 4 CLUSTERS
BMM iteration: 0, error: 0.11183615708017348
BMM iteration: 1, error: 0.02469971349854732
BMM iteration: 2, error: 0.03011391232143803
BMM iteration: 3, error: 0.08173267742854304
BMM iteration: 4, e

BMM iteration: 1, error: 0.02698231438347294
BMM iteration: 2, error: 0.050016409102391336
BMM iteration: 3, error: 0.10176730911879026
BMM iteration: 4, error: 0.08952560271997168
BMM iteration: 5, error: 0.09314699190627405
BMM iteration: 6, error: 0.0582077191171426
BMM iteration: 7, error: 0.03457847951063548
BMM iteration: 8, error: 0.02719921717847935
BMM iteration: 9, error: 0.02166748961636542
BMM iteration: 10, error: 0.017374834968925595
BMM iteration: 11, error: 0.016515011744894885
BMM iteration: 12, error: 0.01751707476443351
BMM iteration: 13, error: 0.01682043147880273
BMM iteration: 14, error: 0.012408343443719655
BMM iteration: 15, error: 0.008076758457798964
BMM iteration: 16, error: 0.005346926126872717
BMM iteration: 17, error: 0.003945548560810945
BMM iteration: 18, error: 0.002448063029098438
BMM iteration: 19, error: 0.0015131660085788998
BMM iteration: 20, error: 0.0010440228267546564
BMM iteration: 21, error: 0.0007570587238693503



TRAINING WITH 5 CLUSTERS
BM

BMM iteration: 17, error: 0.0012468510893180538
BMM iteration: 18, error: 0.0010130572412723467
BMM iteration: 19, error: 0.0008505886547331515



TRAINING WITH 5 CLUSTERS
BMM iteration: 0, error: 0.09231340096872243
BMM iteration: 1, error: 0.019867301717575923
BMM iteration: 2, error: 0.023535813885757224
BMM iteration: 3, error: 0.06781787095858417
BMM iteration: 4, error: 0.09279142305464629
BMM iteration: 5, error: 0.08047499783269908
BMM iteration: 6, error: 0.058813156707993045
BMM iteration: 7, error: 0.043913952470442885
BMM iteration: 8, error: 0.036146828100682364
BMM iteration: 9, error: 0.029384993301660518
BMM iteration: 10, error: 0.021864447790777463
BMM iteration: 11, error: 0.015043584495311292
BMM iteration: 12, error: 0.009447873232219297
BMM iteration: 13, error: 0.006325161209502695
BMM iteration: 14, error: 0.004825783794476244
BMM iteration: 15, error: 0.004035842588400383
BMM iteration: 16, error: 0.003465156550524482
BMM iteration: 17, error: 0.003142659314042

BMM iteration: 9, error: 0.012665550432780167
BMM iteration: 10, error: 0.00790083954904469
BMM iteration: 11, error: 0.0052984118389252455
BMM iteration: 12, error: 0.0038750541490524576
BMM iteration: 13, error: 0.0032800380694730062
BMM iteration: 14, error: 0.003237433089466994
BMM iteration: 15, error: 0.003669556924952888
BMM iteration: 16, error: 0.004093617629162484
BMM iteration: 17, error: 0.004654272007019059
BMM iteration: 18, error: 0.005195892536853237
BMM iteration: 19, error: 0.005480867499628862
BMM iteration: 20, error: 0.004827772152310228
BMM iteration: 21, error: 0.0037280852314290213
BMM iteration: 22, error: 0.0033470862207485687
BMM iteration: 23, error: 0.0031673488291957537
BMM iteration: 24, error: 0.002744880652322081
BMM iteration: 25, error: 0.0021162745268731523
BMM iteration: 26, error: 0.0015180858701459344
BMM iteration: 27, error: 0.0011181896649461894
BMM iteration: 28, error: 0.0008326708405867968



TRAINING WITH 5 CLUSTERS
BMM iteration: 0, error:

BMM iteration: 7, error: 0.042730015381013065
BMM iteration: 8, error: 0.024437615513480775
BMM iteration: 9, error: 0.013657784604175428
BMM iteration: 10, error: 0.0070918865172415035
BMM iteration: 11, error: 0.004070937400600298
BMM iteration: 12, error: 0.0026229106603960282
BMM iteration: 13, error: 0.0017759320900697327
BMM iteration: 14, error: 0.0011928983450472449
BMM iteration: 15, error: 0.0008584756405110388



TRAINING WITH 5 CLUSTERS
BMM iteration: 0, error: 0.09227714130066544
BMM iteration: 7, error: 0.05538492964190094
BMM iteration: 8, error: 0.02967720406533593
BMM iteration: 9, error: 0.014397303548371022
BMM iteration: 10, error: 0.008030002720524487
BMM iteration: 11, error: 0.005377414036596816
BMM iteration: 12, error: 0.004095227000473932
BMM iteration: 13, error: 0.0033529167028692537
BMM iteration: 14, error: 0.0028507024215775434
BMM iteration: 15, error: 0.0027104857197627788
BMM iteration: 16, error: 0.0025880352073267094
BMM iteration: 17, error: 0.00252

BMM iteration: 7, error: 0.0453490235980916
BMM iteration: 8, error: 0.0255632328331191
BMM iteration: 9, error: 0.012587742452247168
BMM iteration: 10, error: 0.0062592568626426625
BMM iteration: 11, error: 0.003582533731992706
BMM iteration: 12, error: 0.002436629683411636
BMM iteration: 13, error: 0.0018504125940769706
BMM iteration: 14, error: 0.0015852856274917667
BMM iteration: 15, error: 0.0014083269075710694
BMM iteration: 16, error: 0.0012527201810999852
BMM iteration: 17, error: 0.0010473513537033882
BMM iteration: 18, error: 0.0006099845864659605



TRAINING WITH 6 CLUSTERS
BMM iteration: 0, error: 0.0777857335977132
BMM iteration: 1, error: 0.016471488347730736
BMM iteration: 2, error: 0.015020760313244622
BMM iteration: 3, error: 0.043153663297762684
BMM iteration: 4, error: 0.08239431123784484
BMM iteration: 5, error: 0.07324448728365113
BMM iteration: 6, error: 0.059686170141666016
BMM iteration: 7, error: 0.04368588502689283
BMM iteration: 8, error: 0.03536716219957263


BMM iteration: 10, error: 0.016605819688859973
BMM iteration: 11, error: 0.014110300012088794
BMM iteration: 12, error: 0.012809129489048407
BMM iteration: 13, error: 0.012238272495091965
BMM iteration: 14, error: 0.01207767589062948
BMM iteration: 15, error: 0.011183554714963403
BMM iteration: 16, error: 0.009267455737864118
BMM iteration: 17, error: 0.007753290087584925
BMM iteration: 18, error: 0.006732295050175987
BMM iteration: 19, error: 0.005527804258259172
BMM iteration: 20, error: 0.004212134447566159
BMM iteration: 21, error: 0.0031057487792752626
BMM iteration: 22, error: 0.0023920374847960957
BMM iteration: 23, error: 0.002023243586547683
BMM iteration: 24, error: 0.0018051813028629312
BMM iteration: 25, error: 0.001670146646610582
BMM iteration: 26, error: 0.0015140059767368974
BMM iteration: 27, error: 0.001203189984901799
BMM iteration: 28, error: 0.0007295079802332078



TRAINING WITH 7 CLUSTERS
BMM iteration: 0, error: 0.06729469850193796
BMM iteration: 1, error: 0.014

BMM iteration: 5, error: 0.0794979447758403
BMM iteration: 6, error: 0.05833872631910507
BMM iteration: 7, error: 0.03852669027413271
BMM iteration: 8, error: 0.023932259962081114
BMM iteration: 9, error: 0.017316616329552507
BMM iteration: 10, error: 0.015282214177324794
BMM iteration: 11, error: 0.013744299606647508
BMM iteration: 12, error: 0.0119798924599118
BMM iteration: 13, error: 0.010022171351442455
BMM iteration: 14, error: 0.0074430897746344225
BMM iteration: 15, error: 0.004944290989026256
BMM iteration: 16, error: 0.00329535256444313
BMM iteration: 17, error: 0.0024725141001374754
BMM iteration: 18, error: 0.0020288929557289413
BMM iteration: 19, error: 0.0017488445039088212
BMM iteration: 20, error: 0.0015787455702197305
BMM iteration: 21, error: 0.00141557331529665
BMM iteration: 22, error: 0.0013382402924528192
BMM iteration: 23, error: 0.0011531469211124214
BMM iteration: 24, error: 0.0009366369304316932



TRAINING WITH 7 CLUSTERS
BMM iteration: 0, error: 0.0672996042

BMM iteration: 8, error: 0.032874720419220224
BMM iteration: 9, error: 0.022316512203103902
BMM iteration: 10, error: 0.015183147866193356
BMM iteration: 11, error: 0.010969038587661593
BMM iteration: 12, error: 0.007983967592804488
BMM iteration: 13, error: 0.0056333662693746995
BMM iteration: 14, error: 0.003541562773025082
BMM iteration: 15, error: 0.0024913942441589353
BMM iteration: 16, error: 0.0019927116962958143
BMM iteration: 17, error: 0.0018613846781585757
BMM iteration: 18, error: 0.0017481512106989213
BMM iteration: 19, error: 0.001747606031378711
BMM iteration: 20, error: 0.0015438074394939446
BMM iteration: 21, error: 0.0014439717493258644
BMM iteration: 22, error: 0.0013261349112625247
BMM iteration: 23, error: 0.0012116596194102467
BMM iteration: 24, error: 0.0011016537792643165
BMM iteration: 25, error: 0.000986917473895322



TRAINING WITH 7 CLUSTERS
BMM iteration: 0, error: 0.06755814054492096
BMM iteration: 1, error: 0.014423706519544114
BMM iteration: 2, error: 0.

BMM iteration: 16, error: 0.01066699827635421
BMM iteration: 17, error: 0.008013290975411168
BMM iteration: 18, error: 0.00572052494321828
BMM iteration: 19, error: 0.003770775946305146
BMM iteration: 20, error: 0.0019095092548201843
BMM iteration: 21, error: 0.000931153173055029



TRAINING WITH 7 CLUSTERS
BMM iteration: 0, error: 0.06725659704490866
BMM iteration: 1, error: 0.013941938714228067
BMM iteration: 2, error: 0.008973786116299967
BMM iteration: 3, error: 0.024120769943961118
BMM iteration: 4, error: 0.06398568020911878
BMM iteration: 5, error: 0.08225831747877228
BMM iteration: 6, error: 0.06289590906866117
BMM iteration: 7, error: 0.045973376626635074
BMM iteration: 8, error: 0.03291865085578425
BMM iteration: 9, error: 0.024550436925764845
BMM iteration: 10, error: 0.01879229474694475
BMM iteration: 11, error: 0.014016509652696131
BMM iteration: 12, error: 0.009757771386684601
BMM iteration: 13, error: 0.007165357298296582
BMM iteration: 14, error: 0.005644086841850841
BM

In [10]:
#PICK THE CLUSTERS

pdi = pd.DataFrame(np.zeros(shape = [len(test_re_B), test_re_B[0].shape[1]]), columns = columns)
pdi_tr = pd.DataFrame(np.zeros(shape = [len(train_re_B), train_re_B[0].shape[1]]), columns = columns)

for i, panda in enumerate(test_re_B):
     pdi.iloc[i,:] = panda.iloc[0,:]
        
for i, panda in enumerate(train_re_B):
     pdi_tr.iloc[i,:] = panda.iloc[0,:]

pdi['seed_number'] = pd.DataFrame(np.repeat(np.array(seed_list_B),7) )
pdi['cluster']= pd.DataFrame(np.tile(np.arange(3,10),10))        



In [11]:
print('Here are the number of clusters and seed_number for the largest auc ')
print(pdi.sort_values(by = 'auc',ascending=False).iloc [0,:][['cluster','auc','seed_number']])
print('Here are the number of clusters and seed_number for the smallest auc ')
print(pdi.sort_values(by = 'auc',ascending=True).iloc [0,:][['cluster','auc','seed_number']])


Here are the number of clusters and seed_number for the largest auc 
cluster           7.000000
auc               0.881126
seed_number    9765.000000
Name: 18, dtype: float64
Here are the number of clusters and seed_number for the smallest auc 
cluster            4.000000
auc                0.875293
seed_number    55375.000000
Name: 29, dtype: float64
